In [387]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from datetime import timedelta, datetime
import os
import math
from random import randrange
import re
import logging

In [1]:
path = "/Users/liruizhi/Downloads/实习/浪潮Inspur/管道压力项目/数据处理"

In [233]:
# 定义函数，读取文件夹下所有文件，输出一个dataframe
def read_folder_files(folder_path):
    output = pd.DataFrame([])
    for file_name in os.listdir(folder_path):
        file_data = pd.read_csv(os.path.join(folder_path, file_name), encoding="gbk")
        output = pd.concat([output, file_data])
    return output

In [230]:
# for f1 in os.listdir(path + "/train_data"):
#     print(f1)
#     fd = pd.read_csv(os.path.join(path + "/train_data", f1), encoding="utf-8")
# #     for index, row in fd.iterrows():
# #         fd.loc[index, "temp_max"] = row["temp_max"].replace("℃","")
# #         fd.loc[index, "temp_min"] = row["temp_min"].replace("℃","")
#     fd.to_csv(os.path.join((path + "\\train_data"), f1), index=None, encoding="gbk")

In [231]:
# for index, row in water.iterrows():
# #         fd.loc[index, "temp_max"] = row["temp_max"].replace("℃","")
#         fd.loc[index, "temp_min"] = row["temp_min"].replace("℃","")
#     fd.to_csv(os.path.join((path + "\\test_data"), f1), index=None, encoding="gbk")

In [238]:
# weather_data_2020 = pd.read_csv(path + "/2020天气.csv")
# for index, row in weather_data_2020.iterrows():
#     weather_data_2020.loc[index, "temp_max"] = row["temp_max"].replace("℃","")
#     weather_data_2020.loc[index, "temp_min"] = row["temp_min"].replace("℃","")
# weather_data_2020.to_csv(path + "/2020天气.csv", encoding="gbk", index=None) 

In [239]:
train_data = read_folder_files(path + "/train_data")
test_data = read_folder_files(path + "/test_data")
to_predict = pd.read_csv(path + "/to_predict.csv", encoding="gbk")

# 读取2020天气数据
weather_data_2020 = pd.read_csv(path + "/2020天气.csv", encoding="gbk")
# 字段名统一
weather_data_2020 = weather_data_2020.rename(columns={"日期":"Time", "天气":"weather"})
test_data = test_data.rename(columns={"天气":"weather"})

# 删除index字段
train_data = train_data.iloc[:, 1:]
test_data = test_data.iloc[:, 1:]
weather_data_2020 = weather_data_2020.iloc[:, 1:]

In [449]:
submit = pd.read_csv(path + "/submit.csv")

In [243]:
# check data和test_data的shape
print("test_data shape:", test_data.shape)
print("train_data shape:", train_data.shape)
print("train_data days number:", train_data["Time"].value_counts().shape)
print("train_data meas number:", train_data["MeasName"].value_counts().shape)

test_data shape: (3720, 31)
train_data shape: (21900, 31)
train_data days number: (730,)
train_data meas number: (30,)


In [244]:
# concat train_data和test_data，以便统一做特征工程
all_data = pd.concat([train_data, test_data, weather_data_2020])
print("all data shape:", all_data.shape)

all data shape: (25934, 31)


D:\program-files\anaconda\envs\env3.6\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [245]:
all_data.head()

,H0,H1,H10,H11,H12,H13,H14,H15,H16,H17,...,H7,H8,H9,MeasName,Time,temp_max,temp_min,weather,wind_dir,wind_level
0,0.288625,0.292000,0.197125,0.224125,0.237250,0.242875,0.257125,0.264625,0.260500,0.23425,...,0.254125,0.225250,0.198250,站点1,2018-01-01,1,-4,多云,东南风,微风
1,0.286750,0.291625,0.209500,0.236125,0.235750,0.247000,0.259375,0.268375,0.274750,0.24625,...,0.217000,0.207250,0.221125,站点1,2018-01-02,8,0,阴转多云,东北风,3-4级
2,0.292000,0.296500,0.235750,0.242875,0.250000,0.241375,0.269125,0.283750,0.258250,0.23800,...,0.240250,0.229750,0.229000,站点1,2018-01-03,1,-1,阴转小雪,东北风,4-5级转4-5级
3,0.290875,0.296500,0.248500,0.214375,0.244000,0.255625,0.253000,0.266875,0.263875,0.24025,...,0.229375,0.216625,0.231250,站点1,2018-01-04,0,-4,阴,东北风转北风,3-4级转3-4级
4,0.284500,0.294625,0.185125,0.203125,0.209125,0.212500,0.229750,0.235000,0.242125,0.22300,...,0.242500,0.199750,0.183625,站点1,2018-01-05,3,-4,阴转多云,西风转北风,3-4级转3-4级


In [246]:
# check whether/wind_dir/wind_level字段的取值范围
print("weather distinct value num:", all_data["weather"].value_counts().shape[0])
print("wind_dir:", all_data["wind_dir"].value_counts().shape[0])
print("wind_level:", all_data["wind_level"].value_counts().shape[0])

weather distinct value num: 72
wind_dir: 67
wind_level: 10


In [247]:
# dummy类别特征
all_data = pd.get_dummies(all_data, columns=["wind_level"], prefix=["wind_level"] )
all_data = pd.get_dummies(all_data, columns=["wind_dir"], prefix=["wind_dir"] )
all_data = pd.get_dummies(all_data, columns=["weather"], prefix=["weather"] )
print("all columns:", all_data.columns.values)

all columns: ['H0' 'H1' 'H10' 'H11' 'H12' 'H13' 'H14' 'H15' 'H16' 'H17' 'H18' 'H19'
 'H2' 'H20' 'H21' 'H22' 'H23' 'H3' 'H4' 'H5' 'H6' 'H7' 'H8' 'H9'
 'MeasName' 'Time' 'temp_max' 'temp_min' 'wind_level_3-4级'
 'wind_level_3-4级转3-4级' 'wind_level_4-5级' 'wind_level_4-5级转4-5级'
 'wind_level_5-6级转5-6级' 'wind_level_6-7级转6-7级' 'wind_level_<3级'
 'wind_level_<3级转<3级' 'wind_level_微风' 'wind_level_转' 'wind_dir_东北风'
 'wind_dir_东北风转东南风' 'wind_dir_东北风转东风' 'wind_dir_东北风转北风' 'wind_dir_东北风转南风'
 'wind_dir_东北风转西北风' 'wind_dir_东北风转西南风' 'wind_dir_东北风转西风' 'wind_dir_东南风'
 'wind_dir_东南风转东北风' 'wind_dir_东南风转东风' 'wind_dir_东南风转北风' 'wind_dir_东南风转南风'
 'wind_dir_东南风转西南风' 'wind_dir_东南风转西风' 'wind_dir_东风' 'wind_dir_东风转东北风'
 'wind_dir_东风转东南风' 'wind_dir_东风转北风' 'wind_dir_东风转南风' 'wind_dir_东风转西北风'
 'wind_dir_东风转西南风' 'wind_dir_东风转西风' 'wind_dir_北风' 'wind_dir_北风转东北风'
 'wind_dir_北风转东南风' 'wind_dir_北风转东风' 'wind_dir_北风转南风' 'wind_dir_北风转微风'
 'wind_dir_北风转西北风' 'wind_dir_北风转西南风' 'wind_dir_北风转西风' 'wind_dir_南风'
 'wind_dir_南风转东北风' 'wind_dir

In [248]:
# 转换Time字段类型为datetime
all_data["Time"] = pd.to_datetime(all_data["Time"])

In [249]:
train_data = all_data.iloc[:train_data.shape[0], :]
test_data = all_data.iloc[train_data.shape[0]:(train_data.shape[0]+ test_data.shape[0]), :]
weather_data_2020 = all_data.iloc[(train_data.shape[0]+ test_data.shape[0]):, :]
print("train_data shape:", train_data.shape)
print("test_data shape:", test_data.shape)
print("weather_data_2020 shape:", weather_data_2020.shape)

train_data shape: (21900, 177)
test_data shape: (3720, 177)
weather_data_2020 shape: (314, 177)


In [250]:
# 暂时考虑只用站点1数据做训练
train_data_1 = train_data[train_data["MeasName"]=="站点1"]
# 去掉Time和MeasName列，方便构造X，Y
train_data_1 = train_data_1.drop(["MeasName", "Time"], axis=1)

In [11]:
# train_num = int(train_data_1.shape[0]*0.8) - 1
# data_train = train_data_1.loc[:train_num, :].reset_index()
# data_val = train_data_1.loc[train_num:, :].reset_index()

In [252]:
def get_hour_data(dataset,
              hour_column,  # the hour column, like H0, H1...
              start_index=0,   # the earliest time to be included in all the historical data for forecasting.
              end_index=None,   # the latest time to be included in all the historical data for forecasting.
              history_length=30,  # the number of timesteps to look back for each forecasting. 
              step_size=1,   # the stride of the history window.
              target_step=0   # the number of periods in the future to predict.
             ):
    # get data for a special hour
    assert step_size > 0
    assert start_index >= 0
    assert hour_column is not None
    
    all_cols = dataset.columns.values.tolist()
    hour_cols = ["H{}".format(i) for i in range(24)]
    feature_cols = [col for col in all_cols if col not in hour_cols]
    all_feature_cols = [hour_column] + feature_cols
    
    if end_index is None:
        end_index = len(dataset) - target_step - history_length
        
    result_x = []
    result_y = []
    for index in range(start_index, end_index):
        row_x = dataset.loc[index: index+history_length-1, all_feature_cols].values
        row_y = dataset.loc[index+history_length, hour_column]
        result_x.append(row_x)
        result_y.append(row_y)
    return (np.array(result_x), np.array(result_y))

In [253]:
def data_load(dataset,
              hour_column=None,  # the hour column, like H0, H1...
              start_index=0,   # the earliest time to be included in all the historical data for forecasting.
              end_index=None,   # the latest time to be included in all the historical data for forecasting.
              history_length=30,  # the number of timesteps to look back for each forecasting. 
              step_size=1,   # the stride of the history window.
              target_step=0   # the number of periods in the future to predict.
             ):
    # load data
    feature_data = []
    label_data = []
    if hour_column is None:
        hour_cols = ["H{}".format(i) for i in range(24) if i not in [11, 13]]  #之所以排除11,13数据，因为这两列数据有问题，计算出损失特别大
        for hour_col in hour_cols:
            data_x, data_y = get_hour_data(dataset=dataset, hour_column=hour_col, start_index=start_index, end_index=end_index, 
                                           history_length=history_length, step_size=step_size, target_step=target_step)
            feature_data.extend(data_x)
            label_data.extend(data_y)
    else:
        data_x, data_y = get_hour_data(dataset=dataset, hour_column=hour_column, start_index=start_index, end_index=end_index, 
                                           history_length=history_length, step_size=step_size, target_step=target_step)
        feature_data.extend(data_x)
        label_data.extend(data_y)
    return (np.asarray(feature_data).astype('float32'), np.array(label_data))

In [254]:
train_data_1.shape[0]

730

In [255]:
HISTORY_LENGTH = 30

trainX, trainY = data_load(train_data_1.iloc[:int(train_data_1.shape[0]*0.8), :])  
# 之所以用前80%数据训练，是因为用全部数据，损失特别大，后面20%数据可能有问题
# 将trainX内部数据格式object转化为float32，以便在keras中可以识别
# trainX = np.asarray(trainX).astype('float32')
print("trainX shape", trainX.shape)
print("trainY shape", trainY.shape)

trainX shape (12188, 30, 152)
trainY shape (12188,)


In [256]:
# create lstm model with keras
HIDDEN_DIMENSION = 10
DROP_RATE = 0.2
LEARNING_RATE = 0.01

inputs = tf.keras.Input(shape = (trainX.shape[1], trainX.shape[2]))
x = tf.keras.layers.LSTM(units=HIDDEN_DIMENSION)(inputs)
x = tf.keras.layers.Dropout(DROP_RATE)(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE),
             loss = tf.keras.losses.MeanSquaredError(),
             metrics=['mse'])

In [257]:
BATCH_SIZE = 128
NUM_EPOCHS = 100

for epoch in range(NUM_EPOCHS):
    print("epoch:{}".format(epoch))
    # model.fit(x=trainX[start:end], y=trainY[start:end], batch_size=BATCH_SIZE)
    model.fit(x=trainX, y=trainY, batch_size=BATCH_SIZE)

epoch:0
Train on 12188 samples
12188/12188 [==============================] - 3s 222us/sample - loss: 0.0320 - mse: 0.0320

In [258]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 30, 152)]         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 10)                6520      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 6,531
Trainable params: 6,531
Non-trainable params: 0
_________________________________________________________________


In [259]:
# 用站点2做验证
train_data_2 = train_data[train_data["MeasName"] == "站点2"].drop(["MeasName", "Time"], axis=1)
eva_x, eva_y = data_load(train_data_2)  
print(eva_x.shape)
model.evaluate(x=eva_x, y=eva_y, batch_size=BATCH_SIZE)

(15400, 30, 152)
15400/1 [==============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[649340.9452558439, 649340.94]

In [23]:
"""
cal mse
idx = 11
aa = trainX[554*idx:554*(idx+1), :, :]
bb = trainY[554*idx:554*(idx+1)]
aay = model(aa).numpy()
mse = np.mean((aay - bb)**2)
"""

'\ncal mse\nidx = 11\naa = trainX[554*idx:554*(idx+1), :, :]\nbb = trainY[554*idx:554*(idx+1)]\naay = model(aa).numpy()\nmse = np.mean((aay - bb)**2)\n'

In [443]:
# 按照月份拆分测试集
test_data["month"] = pd.DatetimeIndex(test_data['Time']).month
month_data_1 = test_data[test_data["month"] == 1]
month_data_3 = test_data[test_data["month"] == 3]
month_data_5 = test_data[test_data["month"] == 5]
month_data_8 = test_data[test_data["month"] == 8]

D:\program-files\anaconda\envs\env3.6\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [353]:
# 预测数据Time转换
to_predict["Time"] = pd.to_datetime(to_predict["Time"])
to_predict["month"] = pd.DatetimeIndex(to_predict['Time']).month
to_predict["day"] = pd.DatetimeIndex(to_predict['Time']).day

In [263]:
# 预测时间
PREDICT_TIME_1 = (datetime(2020, 2, 3, 0, 0), datetime(2020, 2, 16, 0, 0))
PREDICT_TIME_3 = (datetime(2020, 4, 6, 0, 0), datetime(2020, 4, 19, 0, 0))
PREDICT_TIME_5 = (datetime(2020, 6, 1, 0, 0), datetime(2020, 6, 14, 0, 0))
PREDICT_TIME_8 = (datetime(2020, 9, 7, 0, 0), datetime(2020, 9, 20, 0, 0))

In [454]:
# generate result
hour_cols = ["H{}".format(i) for i in range(24)]
meas = ["站点{}".format(i+1) for i in range(30)]
# 构造只保留天气、气温、风力、风向的数据，循环中用到
test_data_aux_cols = [col for col in test_data.columns.values if col not in (hour_cols + ["Time", "MeasName", "month"])]
# 定义起始时间
start_times = [datetime(2020, 1, 31), datetime(2020, 3, 31), datetime(2020, 5, 31), datetime(2020, 8, 31)]
predict_times = [PREDICT_TIME_1, PREDICT_TIME_3, PREDICT_TIME_5, PREDICT_TIME_8]
month_datas = [month_data_1, month_data_3, month_data_5, month_data_8]

for predict_time, month_data, start_time in zip(predict_times, month_datas, start_times):
    
    print("start predict month:{}".format(start_time.month + 1))

    to_predict_day_range = range((predict_time[0] - start_time).days, 
                                 (predict_time[1] - start_time).days+1)
    for mea in meas:
        print("predicting month:{}, mea:{}".format(start_time.month + 1, mea))
        for hour_col in hour_cols:
            # 31是每个月天数        
            test_x = month_data[month_data["MeasName"]==mea].reset_index().loc[(31-HISTORY_LENGTH):, [hour_col]+test_data_aux_cols]
            test_x = np.asarray(test_x).astype("float32").reshape(1, HISTORY_LENGTH, -1)
            for day in range((predict_time[1] - start_time).days):
                # 减t0，加tn-1,去dataframe中查，构造x
                pred_date = (start_time + timedelta(day+1))
                pred_y = model(test_x).numpy()[0][0]
                # 赋值到to_predict
                if day+1 in to_predict_day_range:
                    to_predict.loc[(to_predict["Time"]==pred_date)&(to_predict["MeasName"]==mea)&(to_predict["Hour"]==hour_col), "pred_y"] = pred_y
                test_x = test_x[:, 1:, :]
                last_day_h = np.asarray([pred_y])
                last_day_weather = weather_data_2020[weather_data_2020["Time"]==pred_date][test_data_aux_cols].values[0]
                last_day_x = np.concatenate([last_day_h, last_day_weather]).reshape(1,1,-1)
                test_x = np.concatenate([test_x, last_day_x], axis=1)
            
            
            


start predict month:2
start predict month:2, mea:站点1
start predict month:2, mea:站点2
start predict month:2, mea:站点3
start predict month:2, mea:站点4
start predict month:2, mea:站点5
start predict month:2, mea:站点6
start predict month:2, mea:站点7
start predict month:2, mea:站点8
start predict month:2, mea:站点9
start predict month:2, mea:站点10
start predict month:2, mea:站点11
start predict month:2, mea:站点12
start predict month:2, mea:站点13
start predict month:2, mea:站点14
start predict month:2, mea:站点15
start predict month:2, mea:站点16
start predict month:2, mea:站点17
start predict month:2, mea:站点18
start predict month:2, mea:站点19
start predict month:2, mea:站点20
start predict month:2, mea:站点21
start predict month:2, mea:站点22
start predict month:2, mea:站点23
start predict month:2, mea:站点24
start predict month:2, mea:站点25
start predict month:2, mea:站点26
start predict month:2, mea:站点27
start predict month:2, mea:站点28
start predict month:2, mea:站点29
start predict month:2, mea:站点30
start predict month:4
start

In [463]:
# store result
my_submit = to_predict[["id", "pred_y"]].rename(columns={"pred_y":"pressure"})
to_predict.to_csv(path + "\\result1.csv", encoding="gbk", index=None)
my_submit.to_csv(path + "\\submit1229.csv", index=None)